<a href="https://colab.research.google.com/github/sahithir27/markdown-to-gdoc-automation/blob/main/markdown_to_gdoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build

# Define the markdown content
markdown_content = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""






In [ ]:
# Create a new Google Doc
docs_service = build('docs', 'v1')
document = docs_service.documents().create(body={"title": 'Meeting Notes - May 15, 2024'}).execute()
document_id = document.get("documentId")

In [ ]:
def parse_markdown(markdown):
    """Parse markdown content and return a list of requests for Google Docs API."""
    requests = []
    lines = markdown.split('\n')
    index = 1  # Start index for the document
    i=0
    while i<len(lines):
        line = lines[i]
        # print(document.get('body', {}).get('content', []))
        if not line.strip():
            # Skip empty lines
            i+=1
            continue
        indent_level = (len(line) - len(line.lstrip())) // 2
        if line.startswith('# '):
            requests.extend(create_heading_requests(line[2:], 1, index))
            index += len(line[2:]) + 1  # Update index for next insertion
            print('#', index)
        elif line.startswith('## '):
            requests.extend(create_heading_requests(line[3:], 2, index))
            index += len(line[3:]) + 1
            print('##', index)
        elif line.startswith('### '):
            requests.extend(create_heading_requests(line[4:], 3, index))
            index += len(line[4:]) + 1
            print('###', index)
        elif line.startswith('- [ ]'):
            requests.extend(create_checkbox_requests(line[5:].strip(), index))
            index += len(line[5:].strip()) + 1
            print('- [ ]', index)
        elif line.startswith('- '):
            requests.extend(create_bullet_requests(line[2:], index))
            index += len(line[2:])+1
            print('-', index)
        elif line.lstrip().startswith('* '):
            text=''
            ind_len=0
            while line.lstrip().startswith('* '):
                indetation="\t"*(len(line)-len(line.lstrip()))
                ind_len+=len(indetation)
                line=indetation+line.lstrip().lstrip('*').strip()
                text+=line+'\n'
                print("text_len",len(text))
                i+=1
                if i==len(lines):
                    break
                line=lines[i]
            print(text)
            i=i-1
            requests.extend(createRequestBody_(index, "BULLET_DISC_CIRCLE_SQUARE", text))
            document_length = document.get('body', {}).get('content', [])[-1]['endIndex']
            index += len(text) - ind_len
            print("document_length", document_length)
            print('*', index)
        elif line.startswith('@'):
            requests.extend(create_assignee_requests(line, index))
            index += len(line) + 1
            print('@', index)
        elif line.startswith('---'):
            horizontal_line = "_" * 50 + "\n"
            requests.extend(create_horizontal_line_requests(line, index, horizontal_line))
            index += len(horizontal_line)
        else:
            requests.extend(create_text_requests(line, index))
            index += len(line) + 1
        i+=1
    return requests

def createRequestBody_(start_index, bullet_preset, text):

  return [
      {"insertText": {"text": text, "location": {"index": start_index}}},
      {"createParagraphBullets": {"range": {"startIndex": start_index, "endIndex": start_index + len(text)}, "bulletPreset": bullet_preset}}
  ]



def create_heading_requests(text, level, index):
    """Create requests for a heading."""
    return [
        {
            'insertText': {
                'location': {
                    'index': index
                },
                'text': text + '\n'
            }
        },
        {
            'updateParagraphStyle': {
                'range': {
                    'startIndex': index,
                    'endIndex': index + len(text) + 1
                },
                'paragraphStyle': {
                    'namedStyleType': f'HEADING_{level}'
                },
                'fields': 'namedStyleType'
            }
        }
    ]

def create_bullet_requests(text, index):
    """Create requests for a bullet point."""


    return [
        {
            'insertText': {
                'location': {
                    'index': index
                },
                'text': text + '\n'
            }
        },
        {
            'createParagraphBullets': {
                'range': {
                    'startIndex': index,
                    'endIndex': index + len(text)+1
                },
                'bulletPreset': 'BULLET_DISC_CIRCLE_SQUARE'
            }
        }
    ]

def create_checkbox_requests(text, index):
    """Create requests for a checkbox."""
    return [
        {
            'insertText': {
                'location': {
                    'index': index
                },
                'text': text + '\n'
            }
        },
        {
            'createParagraphBullets': {
                'range': {
                    'startIndex': index,
                    'endIndex': index + len(text) + 1
                },
                'bulletPreset': 'BULLET_CHECKBOX'
            }
        }
    ]

def create_assignee_requests(text, index):
    """Create requests for an assignee mention."""
    return [
        {
            'insertText': {
                'location': {
                    'index': index
                },
                'text': text + '\n'
            }
        },
        {
            'updateTextStyle': {
                'range': {
                    'startIndex': index,
                    'endIndex': index + len(text) + 1
                },
                'textStyle': {
                    'bold': True
                },
                'fields': 'bold'
            }
        }
    ]

def create_horizontal_line_requests(text, index, horizontal_line):
    """Create requests for a horizontal line."""
      # Simulates a horizontal rule


    return [
        {
            "insertText": {
                "location": {"index": index},  # Insert at the start of the document
                "text": horizontal_line
            }
        },
        {
            "updateTextStyle": {
                "range": {
                    "startIndex": index,
                    "endIndex": index + len(horizontal_line)  # Apply styling to the inserted text
                },
                "textStyle": {
                    "bold": False
                },
                "fields": "bold"
            }
        }
    ]

def create_text_requests(text, index):
    """Create requests for plain text."""
    return [
        {
            'insertText': {
                'location': {
                    'index': index
                },
                'text': text+' '
            }
        }
    ]

# Parse the markdown content into requests
requests = parse_markdown(markdown_content)

# Add the formatted text to the Google Doc
result = docs_service.documents().batchUpdate(
    documentId=document_id, body={'requests': requests}).execute()

print(f"Document created: https://docs.google.com/document/d/{document_id}")